In [11]:
import tushare as ts
ts.set_token('00e414e1e675fe57036110eb1f115cb501a6a418a5278b261b86f948')
df = ts.realtime_quote(ts_code='600900.SH')
print(df)
df.to_csv('realtime_data.csv', index=False,encoding='utf-8-sig')

   NAME    TS_CODE      DATE      TIME  OPEN  PRE_CLOSE  PRICE   HIGH    LOW  \
0  长江电力  600900.SH  20250611  15:00:03  30.0      30.03  30.18  30.28  29.98   

     BID  ...  A1_V   A1_P  A2_V  A2_P  A3_V   A3_P  A4_V   A4_P  A5_V   A5_P  
0  30.18  ...    24  30.19   362  30.2   198  30.21  1909  30.22   834  30.23  

[1 rows x 33 columns]


In [27]:
import tushare as ts
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, time
import time as time_module
import threading
import warnings
warnings.filterwarnings('ignore')

# 设置tushare token
ts.set_token('00e414e1e675fe57036110eb1f115cb501a6a418a5278b261b86f948')

def get_realtime_data(stock_code):
    """获取实时数据
    
    Args:
        stock_code: 股票代码，如'600900'(不需要.SH后缀)
    """
    try:
        # 对于tushare的realtime_quote函数，需要去掉市场后缀(.SH/.SZ)
        clean_code = stock_code.split('.')[0]
        
        # 获取实时行情，正确的参数是code而不是ts_code
        df = ts.get_realtime_quotes(clean_code)
        
        if not df.empty:
            current_time = datetime.now()
            df['timestamp'] = current_time
            df['time_str'] = current_time.strftime('%H:%M:%S')
            print(f"成功获取 {stock_code} 实时数据")
            return df
    except Exception as e:
        print(f"获取实时数据失败: {e}")
    return None

# 测试获取数据
df = get_realtime_data('600900.SH')
print("\n数据前5列:")
print(df.iloc[:, :5]) if df is not None else print("没有获取到数据")


成功获取 600900.SH 实时数据

数据前5列:
   name    open pre_close   price    high
0  长江电力  29.980    29.980  30.030  30.290


In [ ]:
import tushare as ts
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, time
import time as time_module
import threading
import warnings
warnings.filterwarnings('ignore')

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

class RealTimeStockAnalyzer:
    def __init__(self, stock_code='600900.SH'):
        # 设置tushare token
        ts.set_token('00e414e1e675fe57036110eb1f115cb501a6a418a5278b261b86f948')
        self.stock_code = stock_code
        # 提取不带后缀的代码，用于tushare的realtime_quotes函数
        self.clean_code = stock_code.split('.')[0]
        
        # 存储实时数据
        self.realtime_data = pd.DataFrame()
        self.daily_high = 0
        self.buy_signals = []
        
        # 交易时间设置
        self.morning_start = time(9, 30)
        self.morning_end = time(11, 30)
        self.afternoon_start = time(13, 0)
        self.afternoon_end = time(15, 0)
        
        # 控制变量
        self.running = False
        print(f"分析器初始化完成，监控股票: {stock_code}")
    
    def is_trading_time(self):
        """判断是否在交易时间内"""
        now = datetime.now().time()
        return ((self.morning_start <= now <= self.morning_end) or 
                (self.afternoon_start <= now <= self.afternoon_end))
    
    def get_realtime_data(self):
        """获取实时数据"""
        try:
            # 使用正确的函数和参数获取实时行情
            df = ts.get_realtime_quotes(self.clean_code)
            
            if not df.empty:
                # 处理数据类型
                df['price'] = df['price'].astype(float)
                df['volume'] = df['volume'].astype(float)
                
                current_time = datetime.now()
                df['timestamp'] = current_time
                df['time_str'] = current_time.strftime('%H:%M:%S')
                return df
        except Exception as e:
            print(f"获取实时数据失败: {e}")
        return None
    
    def calculate_moving_average(self, window=20):
        """计算分时均线"""
        if len(self.realtime_data) >= window:
            return self.realtime_data['price'].rolling(window=window).mean().iloc[-1]
        return None
    
    def update_daily_high(self, current_price):
        """更新当日最高价"""
        if current_price > self.daily_high:
            self.daily_high = current_price
            return True
        return False
    
    def check_buy_conditions(self, current_data):
        """检查买点条件"""
        if len(self.realtime_data) < 20:  # 需要足够的数据计算均线
            return False
        
        current_price = float(current_data['price'].iloc[0])
        current_time = datetime.now()
        
        # 计算分时均线
        ma_20 = self.calculate_moving_average(20)
        if ma_20 is None:
            return False
        
        # 条件1: 股价在分时均线上
        condition1 = current_price > ma_20
        
        # 条件2: 2:30后股价创当天新高
        condition2 = False
        if current_time.time() >= time(14, 30):
            condition2 = self.update_daily_high(current_price)
        
        # 条件3: 回踩不跌破均线 (检查最近5分钟的数据)
        condition3 = self.check_pullback_support(ma_20)
        
        # 记录信号
        signal_info = {
            'time': current_time,
            'price': current_price,
            'ma_20': ma_20,
            'condition1': condition1,
            'condition2': condition2,
            'condition3': condition3,
            'buy_signal': condition1 and (condition2 or condition3)
        }
        
        return signal_info
    
    def check_pullback_support(self, ma_20):
        """检查回踩时是否跌破均线"""
        if len(self.realtime_data) < 10:
            return False
        
        recent_data = self.realtime_data.tail(10)
        recent_prices = recent_data['price'].astype(float)
        
        # 检查是否有回踩（先跌后涨的模式）
        price_changes = recent_prices.diff()
        
        # 找到最低点
        min_price_idx = recent_prices.idxmin()
        min_price = recent_prices.loc[min_price_idx]
        
        # 检查最低点是否在均线上方
        return min_price >= ma_20 * 0.998  # 允许小幅跌破（0.2%以内）
    
    def plot_realtime_chart(self):
        """绘制实时分时图"""
        if len(self.realtime_data) < 2:
            return
        
        plt.figure(figsize=(15, 10))
        
        # 准备数据
        times = pd.to_datetime(self.realtime_data['timestamp'])
        prices = self.realtime_data['price'].astype(float)
        
        # 计算均线
        ma_20 = self.realtime_data['price'].astype(float).rolling(window=20).mean()
        
        # 绘制价格线
        plt.subplot(2, 1, 1)
        plt.plot(times, prices, 'b-', linewidth=1, label='股价', alpha=0.8)
        plt.plot(times, ma_20, 'r--', linewidth=1, label='20分钟均线', alpha=0.7)
        
        # 标记买点
        for signal in self.buy_signals:
            if signal['buy_signal']:
                plt.scatter(signal['time'], signal['price'], 
                           color='red', s=100, marker='^', 
                           label='买点', zorder=5)
        
        plt.title(f'{self.stock_code} 实时分时图 - {datetime.now().strftime("%Y-%m-%d")}')
        plt.ylabel('价格 (元)')
        plt.legend(loc='best')
        plt.grid(True, alpha=0.3)
        
        # 格式化x轴
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
        plt.gca().xaxis.set_major_locator(mdates.MinuteLocator(interval=30))
        plt.xticks(rotation=45)
        
        # 绘制成交量
        plt.subplot(2, 1, 2)
        volumes = self.realtime_data['volume'].astype(float)
        plt.bar(times, volumes, width=0.0005, alpha=0.6, color='gray')
        plt.ylabel('成交量')
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
        plt.xticks(rotation=45)
        
        plt.tight_layout()
        
        # 保存图表
        filename = f"{self.stock_code.replace('.', '_')}_chart_{datetime.now().strftime('%Y%m%d_%H%M')}.png"
        plt.savefig(filename)
        print(f"分时图已保存: {filename}")
        
        plt.show()
    
    def save_buy_signals_report(self):
        """保存买点分析报告"""
        if not self.buy_signals:
            return
        
        report_df = pd.DataFrame(self.buy_signals)
        filename = f"{self.stock_code.replace('.', '_')}_buy_signals_{datetime.now().strftime('%Y%m%d')}.csv"
        report_df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"买点分析报告已保存: {filename}")
    
    def run_analysis(self):
        """运行实时分析"""
        print(f"开始监控 {self.stock_code}...")
        print("监控条件:")
        print("1. 股价在分时均线上")
        print("2. 2:30后股价创当天新高")
        print("3. 回踩时不跌破均线")
        print("-" * 50)
        
        self.running = True
        
        while self.running:
            # 获取实时数据
            current_data = self.get_realtime_data()
            
            if current_data is not None:
                # 添加到历史数据
                if self.realtime_data.empty:
                    self.realtime_data = current_data.copy()
                else:
                    self.realtime_data = pd.concat([self.realtime_data, current_data], 
                                                 ignore_index=True)
                
                # 检查买点条件
                signal_info = self.check_buy_conditions(current_data)
                
                if signal_info:
                    self.buy_signals.append(signal_info)
                    
                    # 打印实时信息
                    current_price = signal_info['price']
                    ma_20 = signal_info.get('ma_20', 0)
                    
                    print(f"\n时间: {signal_info['time'].strftime('%H:%M:%S')}")
                    print(f"当前价格: {current_price:.2f}")
                    if ma_20:
                        print(f"20分钟均线: {ma_20:.2f}")
                    print(f"价格>均线: {signal_info.get('condition1', False)}")
                    print(f"创新高: {signal_info.get('condition2', False)}")
                    print(f"回踩支撑: {signal_info.get('condition3', False)}")
                    
                    if signal_info.get('buy_signal', False):
                        print("🚀 发现买点! 🚀")
                        print("-" * 30)
                        # 检测到买点时立即绘制图表
                        self.plot_realtime_chart()
            
            # 等待1分钟
            time_module.sleep(60)
    
    def stop_analysis(self):
        """停止分析"""
        self.running = False
        self.save_buy_signals_report()
        print("分析已停止")

# 测试函数 - 单次获取数据
def test_get_data():
    analyzer = RealTimeStockAnalyzer('600900.SH')
    df = analyzer.get_realtime_data()
    print(df)

# 主函数
def main():
    # 创建分析器实例
    analyzer = RealTimeStockAnalyzer('600900.SH')  # 可以更改股票代码
    
    try:
        # 启动实时分析
        analyzer.run_analysis()
    except KeyboardInterrupt:
        print("\n用户中断...")
        analyzer.stop_analysis()
    except Exception as e:
        print(f"程序错误: {e}")
        analyzer.stop_analysis()

if __name__ == "__main__":
    # 测试获取数据
    test_get_data()
    
    # 运行完整分析
    # main()


In [12]:
from datetime import date

# 创建一个日期对象
current_date = date.today()  # 获取当前日期
print("今天的日期是:", current_date)

# 使用 weekday() 方法
day_of_week = current_date.weekday()
print("今天是星期几（数字表示）:", day_of_week)

# 将数字转换为星期名称
days = ["星期一", "星期二", "星期三", "星期四", "星期五", "星期六", "星期日"]
print("今天是:", days[day_of_week])

今天的日期是: 2025-06-12
今天是星期几（数字表示）: 3
今天是: 星期四


In [7]:
import akshare as ak

df = ak.stock_board_industry_name_ths()
print(df)
# df2 =ak.stock_board_cons_ths(symbol="半导体", adjust="qfq")
# print(df2)
df3 = ak.stock_info_global_cls(symbol="全部")
print(df3)

      name    code
0      半导体  881121
1       白酒  881273
2     白色家电  881131
3       保险  881156
4     包装印刷  881138
..     ...     ...
85      证券  881157
86      中药  881141
87    专用设备  881118
88      造纸  881137
89  种植业与林业  881101

[90 rows x 2 columns]
                                                   标题  \
0         25家A股上市公司本周披露并购重组最新公告 博众精工拟4.2亿元收购上海沃典70%股权   
1                                       以军称袭击胡塞武装总参谋长   
2                                   以军预计伊朗将继续发动弹道导弹袭击   
3                                      四川省地震局启动四级应急响应   
4   【电报解读】欧盟委员会称欧盟核能计划需2410亿欧元投资！机构看好核心指标Q值持续提升下，可...   
5                       致7死4伤 山东公布寿光较大中毒事故调查报告 26人被问责   
6                                   以军袭击伊朗首都梅赫拉巴德国际机场   
7                                本轮巴以冲突已致加沙地带55362人死亡   
8                                  6月15日周日《新闻联播》要闻20条   
9                                  浙江152条航线因大风已停航108条   
10                                 以军说袭击80多处伊朗核项目相关目标   
11  【风口研报·行业】AI制药里程碑事件！石药集团与阿斯利康达成潜在50亿美元重磅合作，或是AI...   
12      

爬取相关热点，然后用大模型看看热点与股票板块的相关性，再把相关性的股票代码爬取下来，然后进行实时分析

In [13]:
import requests

url = "https://api.apitube.io/v1/news/everything"

querystring = {"per_page":"50", "api_key":"api_live_dKKVoVoviSJahziiuGNvf7C4Z9GFhP7bZxYJ36Tq8uptbtw"}

response = requests.request("GET", url, params=querystring)

print(response.text)



{"status":"not_ok","code":"","request_id":"78e61e46-d4c8-48d5-a59c-a9acce75d881","errors":[{"status":400,"code":"ER0171","message":"Limit is out of range","links":{"about":"https://docs.apitube.io/platform/news-api/http-response-codes"},"timestamp":"2025-06-15T14:04:25Z"}],"user_input":null}
